# ODM 선적 실적 보기

In [1]:
from tool import *
import plotly.express as px
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [2]:
# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',50)

In [33]:
vendor = 'Quanta'
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)
# Series 모델별 연간/월별 선적 실적 출력
df1 = SR_df.pivot_table('Ship', index=['Country', 'Series'], columns=['Ship Week year', 'Ship Week month'], aggfunc=sum)
ms = monthly_sum(get_sp_from_GSCP_DB('2023-05-08(W19)', 'Final', vendor, -1), ['Country', 'Series'])

qt = pd.concat([df1, ms], axis=1, join='outer').fillna(0).convert_dtypes()
qt

In [39]:
vendor = 'Pegatron'
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)
# Series 모델별 연간/월별 선적 실적 출력
df1 = SR_df.pivot_table('Ship', index=['Country', 'Series'], columns=['Ship Week year', 'Ship Week month'], aggfunc=sum)
ms = monthly_sum(get_sp_from_GSCP_DB('2023-05-08(W19)', 'Final', vendor, -1), ['Country', 'Series'])

pg = pd.concat([df1, ms], axis=1, join='outer').fillna(0).convert_dtypes()
pg

(2018, 12)  (2019, 01)  (2019, 02)  (2019, 03)  (2019, 04)  \
Country Series                                                               
Canada  13U70Q           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   
France  CL600            0           0           0           0           0   
...                    ...         ...         ...         ...         ...   
US      14T990         220         130          50         100          50   
        17U70N           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   

                (2019, 05)  (2019, 06)  (2019, 07)  (2019, 08)  (2019, 09)  \
Country Series                                                               
Canada  13U70Q           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   
France  CL600            0           0           0           0           0   
...                    ...         ...         ...         ...         ...   
US      14T990         140           0         530         300          50   
        17U70N           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   

                (2019, 10)  (2019, 11)  (2019, 12)  (2020, 01)  (2020, 02)  \
Country Series                                                               
Canada  13U70Q           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   
France  CL600            0           0           0           0           0   
...                    ...         ...         ...         ...         ...   
US      14T990         220         440           0           0           0   
        17U70N           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0          37           0           0   

                (2020, 03)  (2020, 04)  (2020, 05)  (2020, 06)  (2020, 07)  \
Country Series                                                               
Canada  13U70Q           0           0           0           0           0   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600            0           0           0           0           0   
France  CL600            0           0           0           0           0   
...                    ...         ...         ...         ...         ...   
US      14T990           0           0           0           0           0   
        17U70N           0           0         100        1120         270   
        17U70P           0           0           0           0           0   
        17U70Q           0           0           0           0           0   
        CL600          313         

In [41]:
qt['Vendor'] = 'Quanta'
pg['Vendor'] = 'Pegatron'

In [43]:
qt.shape, pg.shape

((86, 69), (57, 69))

In [45]:
pd.concat([qt, pg]).to_clipboard()

# 검색조건 설정

In [25]:
today = datetime.date.today()
vendor = 'Pegatron'
model = '.'
ship_year = '2023'
ship_month = '.'
buyer= '.'
week = '.'
region = '.'
country = '.'
bl = '.'
bl_status = '.'

In [26]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)
SR_df.reset_index(inplace=True)

In [27]:
cond1 = SR_df['Mapping Model.Suffix'].str.contains(model, na=True)
cond2 = SR_df['Ship Week year'].astype(str).str.contains(ship_year, na=True)
cond3 = SR_df['Ship Month'].astype(str).str.contains(ship_month, na=True)
cond4 = SR_df['Ship To'].str.contains(buyer, na=True)
cond5 = SR_df['Week Name'].str.contains(week)
cond6 = SR_df['Country'].str.contains(country, na=True)
cond7 = SR_df['BL No'].astype(str).str.contains(bl, na=True)
cond8 = SR_df['Region'].str.contains(region, na=True)
cond9 = SR_df['BL Status'].str.contains(bl_status, na=True)

In [28]:
df2 = SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].pivot_table('Ship', index=['Series', 'Country'], columns=['Ship Year', 'Ship Month'], aggfunc=sum, margins=True)

In [29]:
pd.concat([df1, df2], axis=1).to_clipboard()

## 임시함수 작성

In [14]:
SR_df.pivot_table('Ship', index='Region', columns='Ship Week year', aggfunc=sum).sum()

Ship Week year
2020    12704.0
2021    15770.0
2022     7327.0
dtype: float64

In [28]:
SR_df['Country'] = SR_df['Ship To'].replace(site_map)

In [36]:
SR_df['Region'] = SR_df['Country'].replace(country_map)

In [6]:
SR_df.pivot_table('Ship', index='Ship Year', columns='Ship Month', aggfunc=sum).fillna(0)

Ship Month,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
Ship Year,,,,,,,,,,,,
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1319.0
2019,29543.0,15686.0,12617.0,10896.0,10463.0,7884.0,10952.0,15000.0,9729.0,14555.0,8732.0,17797.0
2020,28571.0,2738.0,32710.0,25615.0,19453.0,15625.0,19132.0,16710.0,13688.0,16971.0,13970.0,25400.0
2021,29621.0,24520.0,24586.0,27653.0,29647.0,16841.0,17099.0,9825.0,13051.0,18276.0,27419.0,37251.0
2022,40539.0,24541.0,36531.0,9826.0,13939.0,20510.0,13595.0,13720.0,12352.0,8961.0,8041.0,6657.0
2023,14426.0,14645.0,19493.0,11379.0,9034.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# 매출화 요청시 대상 리스트 출력
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9].pivot_table(
    'Ship', index=['Country', 'Ship To', 'Mapping Model.Suffix', 'PO No.', 'BL No', 'Ship Date', 'PO1 No', 'SO No.', 'Final Destination', 'BL Status'],
    aggfunc=sum)

Ship
Country    Ship To  Mapping Model.Suffix PO No.       BL No      Ship Date  PO1 No  SO No.    Final Destination BL Status         
Austrailia EHAP_LGB 16T90Q-G.AA55A       JPCZ23000911 UTOC500602 2023-02-26 1101566 380160100 AUBNE             CC Requested   5.0
                    16T90Q-G.AA78A       JPCZ22012237 UTOC500568 2023-01-14 1100632 380157554 AUBNE             CC Requested   5.0
                                         JPCZ23000912 UTOC500602 2023-02-26 1101566 380160100 AUBNE             CC Requested   5.0
           EHAP_LGM 16T90Q-G.AA78A       JPCZ23000913 UTOC500593 2023-02-23 1101567 380160101 AUMEL             CC Requested   5.0
           EHAP_LGS 16T90Q-G.AA55A       JPCZ22012556 UTOC500567 2023-01-14 1100895 380158272 AUSYD             CC Requested   5.0
...                                                                                                                            ...
US         ENUS_V   16T90Q-K.APB7U1      JPCZ23001800 UTOC500636 2023-04-01 1746979 380161266 USSBT             CC Requested  20.0
                                         JPCZ23001945 UTOC500632 2023-03-29 1751739 380161926 USSBT             CC Requested  15.0
                                                      UTOC500636 2023-04-01 1751739 380161926 USSBT             CC Requested  15.0
                                                      UTOC500641 2023-04-01 1751739 380161926 USSBT             CC Requested   5.0
                                                      UTOC500645 2023-04-09 1751739 380161926 USSBT             CC Requested  15.0

[133 rows x 1 columns]

In [20]:
color = get_pdr()
color

,Series,Model,Model.Suffix,Marketing Spec.,PDR No.,Grade,Product Variance,Bar Code,Project Code,PDR Description,BASE UNIT,MECHA ASSY,LCD ASSY,WIRELESS LAN,KEYBOARD,BATTERY,OS LANGUAGE,LAN GENDER,MANUAL,WARRANTY,PEN,ADAPTER,POWER CORD,OS TYPE,ACCESSORY KIT,PACKING,EMERGING MARKET,OS DIGITAL KEY,3.0 CPU LABEL,6.0 ENERGY STAR LABEL,1.0 ID LABEL,12. WINDOWS LABEL,BAG,CUSTOMER,USB MOUSE,MASTER,1.1 MAIN BOARD,1.2 SET COLOR,2.4 DTS MARK,1.4 HDMI MARK,Revision,PDR Type,Nation,Shipping Date,Affiliate Code,Production Type,Customer Model,Disabled Status,Bill to Customer,Producing Center,Model Description,Sales Model.Suffix,Sales Affiliate Code,1.5 SET KCC INFORM,1.6 MOUSE KCC INFORM,2.6 PEN KCC INFORM,FINGER PRINT,SSD,SSD2,POUCH,HDMI GENDER (USB TYPE-C),"9.2 INTEL IRIS LABEL(Plus,Xe)",2.0 COUNTRY_BASE BOX LABEL,Image ID,OFFICE KEY CARD,23. POWER RESET LABEL,32. ALEXA LABEL,3.0 CAUTION LABEL,HDMI TO VGA GENDER,BOX ALEXA LABEL,1st Production Date,15.0 CHINA ENERGY LABEL,19.0 CHINA CCC LABEL,1.0 CONFIG LABEL,OFFICE DIGITAL KEY,21.0 DOWNGRADE FR LABEL,C to C Cable,6.1 VIET NAM ENERGY LABEL,26. GREEN MARK STICKER FOR LGETT,MEMORY (SODIMM),HDD,21. HDMI LOGO LABEL,9.3 RADEON LABEL,USB DVD,9.0 NVIDIA LABEL,6.0 SAC LABEL FOR BRAZIL PRODUCTION,FAN,11.1 OPTANE LABEL,DA DIGITAL KEY,SPECIAL LABEL,1.0 POP LABEL
0,11T50Q,11TC50Q,11TC50Q-E.AC11U1,Engineering Spec.,PRZ2301146178,1 (D-14),11TC50Q-CTO,195174052600,PJT2021112300000000115513,11TC50Q-E.AC11U1 PDR 요청드립니다\nPower Cord : Kore...,"(SS) CELERON N5100, eMMC 64GB, MEMORY 4GB, Sub...",GRAY (ABQ30166301),"GRAY, LED LCD W/TOUCH (ABQ30167701)","Intel Wi-Fi 6 AX201 (Wi-Fi 6 , 2x2, BT Combo) ...","GRAY, English (ABQ30166502)","2S-1P, 46.97WH, 6100mAH, Li-lon (EAC64804401)",(CHROME OS)(MS) English,NONE,ENGLISH + MS SIGNATURE,WTY : 1YR,WITH OUT,3PIN 45W BLACK ADAPTER (EAY65907601),USA BLACK 3PIN (EAD02428902),Chrome OS,"COBALT 11, USA, B2B, CHROME OS, MANUAL English...",USA (Chrome OS .Without Carring Bag) (AGF30282...,NONE,NONE,NEW_CELERON (MEZ68861201),ENERGY STAR LABEL FOR Chrom Book (MEZ65079602),"USA, INTEL WIFI 6 AX201 (MEZ68989403)",NONE,NONE,LGEUS,WITH OUT,WITH OUT,NONE,GRAY,NO,NO,0,Global,United States,2023-03-01,ECNT,Production,11TC50Q-E.AC11U1,In Use,LGEUS,OutSourcing,cobalt(11TC50Q)(B2C_B2B) Intel UHD Graphics + ...,11TC50Q-E.AC11U1,"ECNT,ENUS,EKHQ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11T50Q,11TG50Q,11TG50Q-C.AC10KN,Engineering Spec.,PRZ2211140218,1 (D-14),11TG50Q-CTO,8806087975116,PJT2021112300000000115513,,"(SS) CELERON N5100, eMMC 128GB, MEMORY 8GB, Su...",GRAY (ABQ30166301),"GRAY, LED LCD W/TOUCH (ABQ30167701)","Intel Wi-Fi 6 AX201 (Wi-Fi 6 , 2x2, BT Combo) ...","GRAY, KOREAN (ABQ30166501)","2S-1P, 46.97WH, 6100mAH, Li-lon (EAC64804401)",(CHROME OS)(KR) KOREAN,NONE,KOREAN,WTY : 1YR,STYLUS PEN ASS'Y (AAA30715801),3PIN 45W BLACK ADAPTER (EAY65907601),KOREA BLACK 3PIN (EAD61928801),Chrome OS,"COBALT 11, B2G, KOREA, CHROME OS, MANUAL KOR, ...",KOR ( Chorme OS / With Bag ) (AGF30282803),NONE,NONE,NEW_CELERON (MEZ68861201),NaN,"KOREA, B2G, INTEL WIFI 6 AX201 (MEZ68989402)",NONE,11INCH COMMON POUCH_SIDE ZIPPER BAG (MAF66413501),B2G 기본모델(+CEU포함),WITH (WHITE) (AFW72969001),WITH OUT,NONE,GRAY,NO,NO,0,Korea,Republic of Korea,2022-11-25,EKHQ,Production,11TG50Q-CC10KN,In Use,LG ELECTRONICS INC.,OutSourcing,cobalt 11_G(11TG50Q) Intel UHD Graphics + Chro...,11TG50Q-C.AC10KN,EKHQ,"11TG50Q, R-R-LGE-11TC50Q","SM-9023, MSIP-REM-PPQ-SM-9023","WBS21F1LUA, R-R-E8H-WBS21F1LUA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11T50Q,11TC50Q,11TC50Q-E.AC11ML,Engineering Spec.,PRZ2205124287,1 (D-14),11TC50Q-CTO,8806091840097,PJT2021112300000000115513,,"(SS) CELERON N5100, eMMC 128GB, MEMORY 8GB, Su...",GRAY (ABQ30166301),"GRAY, LED LCD W/TOUCH (ABQ30167701)","Intel Wi-Fi 6 AX201 (Wi-Fi 6 , 2x2, BT Combo) ...","

In [23]:
color = color[['Model.Suffix', '1.2 SET COLOR']]

Ship Year             2022                                              \
Ship Month             1.0      2.0      3.0     4.0      5.0      6.0   
Series Country                                                           
11T50Q Korea           NaN   1731.0   1019.0     NaN      NaN    951.0   
14T90P China         140.0      NaN      NaN     NaN     55.0      NaN   
       Korea        2690.0   1021.0    634.0   854.0    163.0    519.0   
       Poland         40.0     20.0      NaN     NaN     35.0      NaN   
       Singapore      40.0     70.0      NaN    30.0     25.0      NaN   
       Spain          60.0      NaN     20.0     NaN      NaN      NaN   
       Taiwan        280.0    285.0     85.0     NaN      NaN      NaN   
       UK             30.0      NaN     10.0     NaN      NaN      NaN   
       US             50.0     30.0    245.0    55.0      NaN      NaN   
14T90Q Korea           NaN      NaN      NaN     NaN      NaN    870.0   
       Poland          NaN      NaN      NaN     NaN      NaN      NaN   
       Singapore       NaN      NaN      NaN     NaN      NaN      NaN   
       Spain           NaN      NaN      NaN     NaN      NaN      NaN   
       Taiwan          NaN      NaN      NaN     NaN      NaN      NaN   
       UK              NaN      NaN      NaN     NaN      NaN      NaN   
       US              NaN      NaN      NaN     NaN      NaN    245.0   
15U40Q Korea        7278.0    289.0   9396.0  2426.0   1932.0    733.0   
15U50N Korea         201.0      NaN      NaN   433.0      8.0    586.0   
15U50P Korea       18294.0  13729.0  13926.0  1513.0   5423.0   8342.0   
15U50Q Korea           NaN      NaN      NaN     NaN    166.0    719.0   
15U70P Korea        1186.0   1552.0   1287.0     NaN      NaN    960.0   
       Spain         135.0      NaN     25.0    20.0      NaN     10.0   
16T90P Canada          NaN    110.0      NaN     NaN     63.0      NaN   
       Germany       150.0    200.0     50.0    35.0     10.0      NaN   
       Japan           NaN      NaN      NaN    70.0     15.0    125.0   
       Korea        7316.0   4894.0   9544.0   370.0    100.0    335.0   
       Spain          15.0      NaN     20.0     NaN      NaN      NaN   
       Taiwan          NaN      NaN    145.0     NaN      NaN     10.0   
       UK             20.0      NaN      NaN    20.0      NaN      NaN   
       US           2614.0    610.0    125.0    50.0     69.0     60.0   
16T90Q Austrailia      NaN      NaN      NaN     NaN      NaN      NaN   
       Canada          NaN      NaN      NaN     NaN      NaN      NaN   
       Germany         NaN      NaN      NaN     NaN      NaN      NaN   
       India           NaN      NaN      NaN     NaN      NaN      NaN   
       Japan           NaN      NaN      NaN     NaN      NaN      NaN   
       Korea           NaN      NaN      NaN   960.0   1440.0    455.0   
       Poland          NaN      NaN      NaN     NaN      NaN      NaN   
       Singapore       NaN      NaN      NaN     NaN      NaN      NaN   
       Spain           NaN      NaN      NaN     NaN      NaN      NaN   
       Taiwan          NaN      NaN      NaN     NaN      NaN      NaN   
       UK              NaN      NaN      NaN     NaN      NaN      NaN   
       US              NaN      NaN      NaN  2990.0   4435.0   5590.0   
All                40539.0  24541.0  36531.0  9826.0  13939.0  20510.0   

Ship Year                                                                  All  
Ship Month             7.0      8.0      9.0    10.0    11.0    12.0            
Series Country                                                                  
11T50Q Korea        1239.0    728.0    494.0    43.0     NaN   703.0    6908.0  
14T90P China           NaN      NaN      NaN     NaN     NaN     NaN     195.0  
       Korea         491.0    310.0     85.0     NaN     NaN     NaN    6767.0  
       Poland          NaN      NaN      NaN     NaN     NaN     NaN      95.0  
       Singapore       NaN  

In [7]:
pd.merge(SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].pivot_table('Ship', index=['Country', 'Mapping Model.Suffix' ], columns=['Ship Year', 'Ship Month'], aggfunc=sum, margins=True).reset_index(), color,
         left_on='Mapping Model.Suffix', right_on='Model.Suffix' ).drop('Model.Suffix', axis=1)

NameError: name 'color' is not defined

In [11]:
SR_df.columns

Index(['PO No.', 'BL No', 'Mapping Model.Suffix', 'Ship To', 'Shipping',
       'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship',
       'Issued Date', 'RSD', 'Ship Date', 'BL Status', 'Method', 'Price Term',
       'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
       'Final Destination', 'PO Week', 'PO Month', 'PO Year', 'RSD Week',
       'RSD Month', 'RSD Year', 'RSD Week Year', 'RSD Week Month',
       'RSD Week Day', 'RSD Week Name', 'Country', 'Region', 'Ship Week',
       'Ship Month', 'Ship Year', 'Ship Week month', 'Ship Week day',
       'Ship Week year', 'Week Name'],
      dtype='object')

In [12]:
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].pivot_table('Ship', index=['Country', 'Ship To', 'Mapping Model.Suffix', 'PO No.', 'BL No', 'Ship Date'], columns='Ship Month', aggfunc=sum)

Ship Month                                                                  10.0
Country Ship To Mapping Model.Suffix PO No.       BL No        Ship Date        
Spain   EEES    PH30N.AEUQ           JGMZ22024435 PLISZ2D01194 2022-10-30  100.0
                                     JGMZ22024938 PLISZ2D01194 2022-10-30  100.0

In [16]:
SR_df['Amt'] = SR_df['Unit Price'] * SR_df['Ship']

In [17]:
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].pivot_table('Amt', index='Mapping Model.Suffix', columns=['Ship Year', 'Ship Month', 'Week Name'], aggfunc=sum, margins=True).fillna(0).convert_dtypes()

,2022,All
,10.0,
,2022-10-10(W41),
Mapping Model.Suffix,,
PH30N.AEUQ,9029.72,9029.72
All,9029.72,9029.72


# 조회

In [18]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

In [19]:
SR_df.loc['JPCZ22006821']

KeyError: 'JPCZ22006821'

In [ ]:
SR_df.shape

In [ ]:
SR_df = SR_df.drop(('JPCZ22006821', 'UTOC500381')).shape

In [ ]:
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8].to_clipboard()

In [ ]:
pd.pivot_table(SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8],
              index=['Country', 'Mapping Model.Suffix'], columns = 'Week Name', values = 'Ship',
              aggfunc=sum).fillna('-').to_clipboard()

In [ ]:
pd.pivot_table(SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8],
               index=['BL Status', 'Country', 'Mapping Model.Suffix', 'Ship Date', 'BL No'], columns = ['Week Name'], values = 'Ship', aggfunc=sum).fillna('-').tail(20)

## 주의) 변경된 SR DB를 저장

In [ ]:
with open('D:/Data/Quanta shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)

## 1-1) Quanta 실적

In [ ]:
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

In [ ]:
SR_df.reset_index(inplace=True)

In [ ]:
search_year = '2022'
search_month = '.'
thisweek = '.'
model_name = '.'
site = 'EK'
country = '.'
bl = '22026000'

cond1 = (SR_df['Ship Year'].astype(str).str.contains(search_year, na=True))
cond2 = (SR_df['Ship Week month'].astype(str).str.contains(search_month, na=True))
cond3 = (SR_df['Week Name'].str.contains(thisweek, na=True))
cond4 = (SR_df['Mapping Model.Suffix'].str.contains(model_name, na=True))
cond5 = (SR_df['Ship To'].str.contains(site, na=True))
cond6 = (SR_df['Country'].str.contains(country, na=True))
cond7 = (SR_df['BL No'].astype(str).str.contains(bl, na=True))

In [ ]:
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7]

In [ ]:
# 모델명으로 조회
SR_df[SR_df['Mapping Model.Suffix'] == '11TC50Q-E.AC10ML']

In [ ]:
# 특정 BL 번호로 조회
SR_df.swaplevel(0,1).loc[''].groupby(['Mapping Model.Suffix', 'PO No.', 'Ship Date'])['Ship'].sum().reset_index()

In [ ]:
# 특정 년도, 월의 선적 실적 출력
SR_df[(SR_df['Ship Year'] == 2020) & (SR_df['Ship Month'] == 3)].sort_values(by='Ship Date')

In [ ]:
# 테이블로 출력
pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Series'] == '15U40Q') & (SR_df['Ship Month']<13)], index=['Region', 'Country', 'Ship To', 'Mapping Model.Suffix'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

In [ ]:
# 이번주, 지난 주 선적 실적 보기
today = datetime.date.today()
seven_day_ago = today - datetime.timedelta(days=7)
this_week = f'{getfirstdate_year(today)}-{getfirstdate_month(today)}-{getfirstdate_day(today)}(W{today.isocalendar()[1]:02d})'
last_week = f'{getfirstdate_year(seven_day_ago)}-{getfirstdate_month(seven_day_ago)}-{getfirstdate_day(seven_day_ago)}(W{seven_day_ago.isocalendar()[1]:02d})'
pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Ship Month'] == 3) & (SR_df['Week Name'] == this_week) ], index=['Region', 'BL No', 'BL Status', 'Country', 'Ship To', 'Mapping Model.Suffix', 'PO No.','Ship Date'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

In [ ]:
pdr_df = get_pdr()

In [ ]:
pd.pivot_table(SR_df[SR_df['Ship Year'] == 2022] , index=['Country', 'Ship To', 'Series', 'Mapping Model.Suffix'], columns=['Ship Year', 'Ship Month', 'Week Name'], values=['Ship']).fillna('-').droplevel([0,1,2], axis=1).reset_index()

In [ ]:
# 지역별, 모델별, 월별, 선적실적 보기
pd.merge(pd.pivot_table(SR_df[SR_df['Ship Year'] == 2022] , index=['Country', 'Ship To', 'Series', 'Mapping Model.Suffix'], columns=['Ship Year', 'Ship Month', 'Week Name'], values=['Ship'], aggfunc=sum).fillna('-').droplevel([0,1,2], axis=1).reset_index(), pdr_df, how='left', left_on='Mapping Model.Suffix', right_on='Model.Suffix').to_excel('D:/Python result/res2.xlsx')

## 1-2) Quanta Open PO 구하기

In [ ]:
with open('D:/Data/Quanta PO DB.bin', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df.head(1)

In [ ]:
df.loc['YPCZ21003095']

In [ ]:
df[df['Mapping Model.Suffix'].str.contains('15UD470')]

In [ ]:
df['Mapping Model.Suffix'].unique()

In [ ]:
with open('D:/Data/Quanta PO DB.bin', 'rb') as f:
    PO_df = pickle.load(f)
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
    SR_df = pickle.load(f)

open_po_info = {} # Open PO 정보를 dictionary 형태로 저장
grouped = SR_df.groupby('PO No.')
for i in PO_df.index:
    try:
        shipped_qty = grouped.get_group(i)['Ship'].sum()
    except:
        shipped_qty = 0
    open_po_qty = PO_df.loc[i, 'PO'] - shipped_qty
    if open_po_qty > 0:
        open_po_info[i] = open_po_qty

open_po_df = PO_df.loc[open_po_info.keys()]
open_po_df['open qty'] = open_po_info.values()

In [ ]:
pd.pivot_table(open_po_df[open_po_df['Cancel'] == 'N'], index=['Series', 'Ship To', 'Mapping Model.Suffix'], columns=['RSD Week Name'], values=['open qty'], aggfunc=sum).fillna('-').to_excel('c.xlsx')

## 2-1) Pegatron 실적

In [ ]:
with open('D:/Data/Pegatron shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

In [ ]:
SR_df.columns

In [ ]:
pd.pivot_table(SR_df, index=['Series'], columns=['Ship Year', 'Ship Month'], values='Ship', aggfunc=sum).fillna('-').to_excel('D:/Python result/pegatron ship result.xlsx')

In [ ]:
pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Ship Month']==4)], index=['Region', 'Country', 'Ship To', 'Mapping Model.Suffix', 'BL No'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

In [ ]:
# 이번주, 지난 주 선적 실적 보기
today = datetime.date.today()
seven_day_ago = today - datetime.timedelta(days=7)
this_week = f'{getfirstdate_year(today)}-{getfirstdate_month(today)}-{getfirstdate_day(today)}(W{today.isocalendar()[1]:02d})'
last_week = f'{getfirstdate_year(seven_day_ago)}-{getfirstdate_month(seven_day_ago)}-{getfirstdate_day(seven_day_ago)}(W{seven_day_ago.isocalendar()[1]:02d})'
pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Ship Month'] == 4) & (SR_df['Week Name'] == this_week) ], index=['Region', 'BL No', 'BL Status', 'Country', 'Ship To', 'Mapping Model.Suffix', 'PO No.','Ship Date'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

## 2-2) Pegatron OPEN PO 구하기

In [ ]:
with open('D:/Data/Pegatron PO DB.bin', 'rb') as f:
    PO_df = pickle.load(f)
with open('D:/Data/Pegatron shipment result DB.bin', 'rb') as f:
    SR_df = pickle.load(f)

open_po_info = {} # Open PO 정보를 dictionary 형태로 저장
grouped = SR_df.groupby('PO No.')
for i in PO_df.index:
    try:
        shipped_qty = grouped.get_group(i)['Ship'].sum()
    except:
        shipped_qty = 0
    open_po_qty = PO_df.loc[i, 'PO'] - shipped_qty
    if open_po_qty > 0:
        open_po_info[i] = open_po_qty

open_po_df = PO_df.loc[open_po_info.keys()]
open_po_df['open qty'] = open_po_info.values()
pd.pivot_table(open_po_df[open_po_df['Cancel'] == 'N'], index=['Series', 'Ship To', 'Mapping Model.Suffix'], 
               columns=['RSD Week Name'], values=['open qty'], aggfunc=sum).fillna('-')

## 3-1) Wanlida 실적

In [ ]:
with open('D:/Data/Wanlida shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

## 검색 조건 설정

In [ ]:
today = datetime.date.today()
search_year = '2022'
search_month = '.'
week = '.'
model_name = '.'
site = 'GF'
country = '.'

cond1 = (SR_df['Ship Year'].astype(str).str.contains(search_year, na=True))
cond2 = (SR_df['Ship Week month'].astype(str).str.contains(search_month, na=True))
cond3 = (SR_df['Week Name'].str.contains(week))
cond4 = (SR_df['Mapping Model.Suffix'].str.contains(model_name, na=True))
cond5 = (SR_df['Ship To'].str.contains(site, na=True))
cond6 = (SR_df['Country'].str.contains(country, na=True))

In [ ]:
SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6]

In [ ]:
pd.pivot_table( SR_df[cond1 & cond2 & cond3 & cond4 & cond5 & cond6], index=['BL No', 'Ship Date', 'Final Destination', 'Ship To', 'Mapping Model.Suffix'], values = 'Ship', aggfunc=sum )

In [ ]:

pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Ship Month'] == 4) & (SR_df['Week Name'] == thisweek) ], index=['Region', 'BL No', 'BL Status', 'Country', 'Ship To', 'Mapping Model.Suffix', 'PO No.','Ship Date', 'Unit Price'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

In [ ]:
SR_df.swaplevel(0,1).loc['PLISZ2D00333', ['Region', 'Country', 'Ship To', 'Mapping Model.Suffix', 'Ship', 'PO1 No', 'SO No.', 'Final Destination']]

In [ ]:
pd.pivot_table(SR_df[(SR_df['Ship Year']==2022) & (SR_df['Ship Month']==3)], index=['Region', 'Country', 'Ship To', 'Mapping Model.Suffix', 'BL No'], columns=['Ship Year', 'Ship Month', 'Week Name'], values = 'Ship', aggfunc=sum).fillna('-')

## 3-2) Wanlida OPEN PO 구하기

In [ ]:
open_po_df = get_open_po('Wanlida')
pd.pivot_table(open_po_df[open_po_df['Cancel'] == 'N'], index=['Ship To', 'Mapping Model.Suffix'], 
               columns=['RSD Month', 'RSD Week Name'], values=['Open_Qty'], aggfunc=sum)

In [ ]:
open_po_df[open_po_df['Ship To'].str.contains('GF')]

## 4) PC ODM 월별 선적 실적 보기

In [73]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [74]:
sr = sr[sr['Division']=='PC'] # PC 실적만 조회
sr = sr.reset_index()
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, ste[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
sr['Series'] = sr['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6])
sr = sr[sr['Year']=='2023'] # 2023년 실적으로 조회

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_1216\731353633.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  sr['Series'] = sr['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6])


In [75]:
sr = sr.pivot_table('Ship', index=['From Site', 'Country', 'Series'], columns=['Year', 'Month'], aggfunc=sum).fillna(0)
d = datetime.datetime.today()
filepath_to_save = 'D:/PC_ODM_월별 실적/2023년 PC_ODM_월별 선적 실적_{}.xlsx'.format(d.strftime('%y%m%d'))
sr.to_excel(filepath_to_save, startrow=1)

In [83]:
wb = load_workbook(filepath_to_save)
color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

NameError: name 'load_workbook' is not defined

In [ ]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ {d.strftime('%y%m%d')} PC ODM Shipment Monthly Shipment'
sh['A1'].font = middle_size
sh['C1'] = 'Sub TTL'
for i in range(4, sh.max_column):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
min_row_num = 2
height = len(sr.columns[0])
max_row_num = min_row_num + height_gap_region - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=2, max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
min_row_num = max_row_num + 1
for row in sh.iter_rows(min_row=min_row_num, min_col=1, max_row=min_row_num, max_col=len(sr.index[0])):
    for cell in row:
        cell.fill = color
min_row_num = min_row_num + 1
max_row_num = min_row_num + sr.shape[0] - 1
for row in sh.iter_rows(min_row=min_row_num, min_col=len(sr.index[0]), max_row=max_row_num, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0
        
sh.freeze_panes = 'D5'
sh.column_dimensions['A'].width = 20
sh.column_dimensions['B'].width = 10

In [2]:
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
     SR_df_Quanta = pickle.load(f)
with open('D:/Data/Pegatron shipment result DB.bin', 'rb') as f:
     SR_df_Pegatron = pickle.load(f)
with open('D:/Data/Wingtech shipment result DB.bin', 'rb') as f:
     SR_df_Wingtech = pickle.load(f)
qt_result = pd.pivot_table(SR_df_Quanta[SR_df_Quanta['Ship Year'] == 2023], index=['Country', 'Series'], columns='Ship Month', values='Ship', aggfunc=sum).fillna(0)
pg_result = pd.pivot_table(SR_df_Pegatron[SR_df_Pegatron['Ship Year'] == 2023], index=['Country', 'Series'], columns='Ship Month', values='Ship', aggfunc=sum).fillna(0)
wt_result = pd.pivot_table(SR_df_Wingtech[SR_df_Wingtech['Ship Year'] == 2023], index=['Country', 'Series'], columns='Ship Month', values='Ship', aggfunc=sum).fillna(0)

df = pd.concat([qt_result, pg_result, wt_result]).reset_index()
df.insert(0, 'From Site', df['Series'].replace(vendor_find))
df

Ship Month,From Site,Country,Series,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
0,Quanta,Austrailia,16T90Q,25.0,15.0,7.0,0.0,0.0,0.0,0.0,0.0
1,Quanta,Austrailia,16T90R,0.0,0.0,0.0,210.0,115.0,35.0,0.0,0.0
2,Quanta,Canada,16T90Q,40.0,0.0,40.0,0.0,20.0,20.0,0.0,0.0
3,Quanta,Canada,16T90R,0.0,0.0,5.0,0.0,40.0,40.0,0.0,0.0
4,Quanta,Germany,14T90R,0.0,0.0,5.0,30.0,5.0,50.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
57,Pegatron,UK,CL600,140.0,0.0,760.0,0.0,1255.0,0.0,150.0,NaN
58,Pegatron,US,13U70P,0.0,50.0,0.0,0.0,0.0,0.0,0.0,NaN
59,Pegatron,US,17U70Q,15.0,190.0,0.0,0.0,0.0,0.0,0.0,NaN
60,Pegatron,US,CL600,955.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [3]:
def get_month_name(month_number):
    """Convert a month number (1-12) to the month name."""
    if isinstance(month_number, (int, float)):
        date = datetime.datetime.strptime(f"{int(month_number)}", "%m")
        return date.strftime("%b")
    else:
        return month_number

In [4]:
df.groupby(['From Site', 'Country', 'Series']).sum().columns.map(get_month_name)

Index(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug'], dtype='object', name='Ship Month')

In [5]:
df1 = df.groupby(['From Site', 'Country', 'Series']).sum()
df1.columns = df1.columns.map(get_month_name)
month_type = CategoricalDtype(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)
df1.columns = df1.columns.astype(month_type)
d = datetime.date.today()
df1.loc[:, :get_month_name(d.month-1)].to_excel('D:/PC_ODM_월별 실적/2023년 PC_ODM_월별 선적 실적_{}.xlsx'.format(d.strftime('%y%m%d')))